In [3]:
!pip install gymnasium[box2d]
!pip install stable-baselines3[extra]

In [4]:
import gymnasium as gym
from stable_baselines3 import SAC

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Create Environment

In [5]:
env = gym.make("LunarLanderContinuous-v3")

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/

Test random agent

In [6]:
obs, _ = env.reset()

for _ in range(200):
    action = env.action_space.sample()
    obs, reward, done, trunc, info = env.step(action)
    if done or trunc:
        obs, _ = env.reset()

Train SAC Agent (Baseline)

In [7]:
model = SAC(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    buffer_size=50000
)

model.learn(total_timesteps=50000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 164      |
|    ep_rew_mean     | -331     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 40       |
|    time_elapsed    | 16       |
|    total_timesteps | 658      |
| train/             |          |
|    actor_loss      | 1.06     |
|    critic_loss     | 3.91     |
|    ent_coef        | 0.851    |
|    ent_coef_loss   | -0.48    |
|    learning_rate   | 0.0003   |
|    n_updates       | 557      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 155      |
|    ep_rew_mean     | -371     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 39       |
|    time_elapsed    | 31       |
|    total_timesteps | 1240     |
| train/             |          |
|    actor_loss      | 4.56     |
|    critic_loss     | 3.83     |
|    ent_coef 

In [12]:
model.save("baseline")

Make It Fuel-Efficient

Thrusters consume fuel → penalize large actions.
Create wrapper

In [8]:
import numpy as np
from gymnasium import Wrapper

class FuelEfficientWrapper(Wrapper):
    def step(self, action):
        obs, reward, done, trunc, info = self.env.step(action)

        fuel_penalty = 0.1 * np.sum(np.square(action))
        reward -= fuel_penalty

        return obs, reward, done, trunc, info

In [9]:
env = FuelEfficientWrapper(
    gym.make("LunarLanderContinuous-v3")
)

Train Again with SAC

In [10]:
model = SAC("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=50000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 124      |
|    ep_rew_mean     | -270     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 45       |
|    time_elapsed    | 10       |
|    total_timesteps | 497      |
| train/             |          |
|    actor_loss      | 0.465    |
|    critic_loss     | 85.8     |
|    ent_coef        | 0.894    |
|    ent_coef_loss   | -0.314   |
|    learning_rate   | 0.0003   |
|    n_updates       | 396      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 239      |
|    ep_rew_mean     | -236     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 38       |
|    time_elapsed    | 49       |
|    total_timesteps | 1909     |
| train/             |

In [13]:
model.save("sac_basic")

Record Landing Videos

In [14]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import SAC

def record_landings(model_path, video_folder, fuel_wrapper=False):

    # Environment
    env = gym.make(
        "LunarLanderContinuous-v3",
        render_mode="rgb_array"
    )

    # Apply wrapper ONLY for fuel-efficient model
    if fuel_wrapper:
        env = FuelEfficientWrapper(env)

    env = RecordVideo(
        env,
        video_folder=video_folder,
        episode_trigger=lambda e: e < 4
    )

    model = SAC.load(model_path)

    MAX_STEPS = 1000

    for ep in range(4):
        print(f"Recording Episode {ep+1}")

        obs, _ = env.reset()

        for _ in range(MAX_STEPS):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = env.step(action)

            if terminated or truncated:
                break

    env.close()

Record BASELINE landing

In [15]:
record_landings(
    "baseline",
    "./videos_baseline",
    fuel_wrapper=False
)

Recording Episode 1
Recording Episode 2


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Recording Episode 3


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Recording Episode 4


Record FUEL-EFFICIENT SAC landing

In [16]:
record_landings(
    "sac_basic",
    "./videos_fuel",
    fuel_wrapper=True
)

Recording Episode 1
Recording Episode 2
Recording Episode 3
Recording Episode 4


In [17]:
!ls videos_baseline
!ls videos_fuel

rl-video-episode-0.mp4	rl-video-episode-2.mp4
rl-video-episode-1.mp4	rl-video-episode-3.mp4
rl-video-episode-0.mp4	rl-video-episode-2.mp4
rl-video-episode-1.mp4	rl-video-episode-3.mp4


In [19]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    return HTML(f"""
    <video width=500 controls>
        <source src="{data_url}" type="video/mp4">
    </video>
    """)

Video: Baseline landing (Showing four Attempts)

In [22]:
for i in range(4):
    display(show_video(f"videos_baseline/rl-video-episode-{i}.mp4"))

Video: Fuel Efficient SAC landing (Showing Four Attempts)

In [23]:
for i in range(4):
    display(show_video(f"videos_fuel/rl-video-episode-{i}.mp4"))

Average Episode Reward : How good the learned policy is overall.

In [24]:
import numpy as np
import gymnasium as gym
from stable_baselines3 import SAC

def evaluate_model(model_path, fuel_wrapper=False, episodes=20):

    env = gym.make("LunarLanderContinuous-v3")

    if fuel_wrapper:
        env = FuelEfficientWrapper(env)

    model = SAC.load(model_path)

    rewards = []

    for ep in range(episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = env.step(action)

            total_reward += reward
            done = terminated or truncated

        rewards.append(total_reward)

    env.close()

    return np.mean(rewards), np.std(rewards)

In [25]:
baseline_reward = evaluate_model("baseline", False)
fuel_reward = evaluate_model("sac_basic", True)

print("Baseline:", baseline_reward)
print("Fuel Efficient:", fuel_reward)

Baseline: (np.float64(214.46608789491307), np.float64(105.46337863224572))
Fuel Efficient: (np.float64(156.77402006550204), np.float64(106.02195820774115))


Fuel Consumption Metric

Fuel usage ≈ thrust magnitude.

We compute:

Fuel= ∑ action^2



Fuel Measurement

In [26]:
def evaluate_fuel(model_path, fuel_wrapper=False, episodes=20):

    env = gym.make("LunarLanderContinuous-v3")

    if fuel_wrapper:
        env = FuelEfficientWrapper(env)

    model = SAC.load(model_path)

    fuel_usage = []

    for _ in range(episodes):
        obs, _ = env.reset()
        done = False
        fuel = 0

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            fuel += np.sum(np.square(action))

            obs, _, terminated, truncated, _ = env.step(action)
            done = terminated or truncated

        fuel_usage.append(fuel)

    env.close()

    return np.mean(fuel_usage)

Compare fuel

In [27]:
print("Baseline fuel:", evaluate_fuel("baseline"))
print("Fuel-efficient SAC:", evaluate_fuel("sac_basic", True))

Baseline fuel: 168.61021
Fuel-efficient SAC: 153.7056


Landing Success Rate

In [31]:
def landing_success_rate(model_path, fuel_wrapper=False, episodes=20):

    env = gym.make("LunarLanderContinuous-v3")

    if fuel_wrapper:
        env = FuelEfficientWrapper(env)

    model = SAC.load(model_path)

    success = 0

    for _ in range(episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0   # accumulate reward for entire episode

        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = env.step(action)

            total_reward += reward
            done = terminated or truncated

        if total_reward > 200:   # use total episode reward
            success += 1

    env.close()

    return success / episodes

In [32]:
# Evaluate success rate for baseline
baseline_success = landing_success_rate("baseline", fuel_wrapper=False, episodes=20)
print("Baseline success rate:", baseline_success)

# Evaluate success rate for fuel-efficient SAC
fuel_success = landing_success_rate("sac_basic", fuel_wrapper=True, episodes=20)
print("Fuel-efficient SAC success rate:", fuel_success)

Baseline success rate: 0.55
Fuel-efficient SAC success rate: 0.6


In [33]:
print(f"Baseline success rate: {baseline_success*100:.1f}%")
print(f"Fuel-efficient SAC success rate: {fuel_success*100:.1f}%")

Baseline success rate: 55.0%
Fuel-efficient SAC success rate: 60.0%
